<a href="https://colab.research.google.com/github/DhavalTaunk08/NLP_scripts/blob/master/sentiment_analysis_using_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers==3.0.2

  Using cached transformers-3.0.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.8.1rc1.tar.gz (97 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      /tmp/pip-build-env-fnx7ll_3/overlay/lib/python3.10/site-packages/setuptools/dist.py:294: InformationOnly: Normalizing '0.8.1.rc1' to '0.8.1rc1'
        self.metadata.version = self._normalize_version(self.metadata.version)
      running bdist_wheel
      running build
      running build_py
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying tokenizers/__init__.py -> build

In [2]:


import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split

import torch

import seaborn as sns

import transformers

import json

from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader

from transformers import RobertaModel, RobertaTokenizer

import logging

logging.basicConfig(level=logging.ERROR)

In [4]:

from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
train = pd.read_csv('/kaggle/input/new-data/new_dataset.csv')

In [6]:
train["Class"] = train["Class"].replace({
    'student': 0,
    'researcher': 1,
    'corporates': 2
})

/tmp/ipykernel_30/3921634441.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["Class"] = train["Class"].replace({


In [7]:
train.shape

(441, 2)

In [8]:
train.head()

,Email,Class
0,"Dear Dr. Smith, I am reaching out to inquire a...",1
1,I am writing to request access to the universi...,1
2,I am a Ph.D.student in need of guidance on my ...,1
3,I am interested in collaborating with your res...,1
4,I am having trouble with my research methodolo...,1


In [9]:
train["Class"].unique()

array([1, 0, 2])

In [10]:
new_df = train[['Email', 'Class']]

In [11]:

MAX_LEN = 256

TRAIN_BATCH_SIZE = 2

VALID_BATCH_SIZE = 2

LEARNING_RATE = 1e-05

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
class EmailData(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):

        self.tokenizer = tokenizer

        self.data = dataframe

        self.text = dataframe.Email

        self.targets = self.data.Class

        self.max_len = max_len



    def __len__(self):

        return len(self.text)



    def __getitem__(self, index):

        text = str(self.text[index])

        text = " ".join(text.split())



        inputs = self.tokenizer.encode_plus(

            text,

            None,

            add_special_tokens=True,

            max_length=self.max_len,

            pad_to_max_length=True,

            return_token_type_ids=True

        )

        ids = inputs['input_ids']

        mask = inputs['attention_mask']

        token_type_ids = inputs["token_type_ids"]





        return {

            'ids': torch.tensor(ids, dtype=torch.long),

            'mask': torch.tensor(mask, dtype=torch.long),

            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),

            'targets': torch.tensor(self.targets[index], dtype=torch.float)

        }

In [13]:
train_size = 0.7

train_data=new_df.sample(frac=train_size,random_state=200)

test_data=new_df.drop(train_data.index).reset_index(drop=True)

train_data = train_data.reset_index(drop=True)





print("FULL Dataset: {}".format(new_df.shape))

print("TRAIN Dataset: {}".format(train_data.shape))

print("TEST Dataset: {}".format(test_data.shape))



training_set = EmailData(train_data, tokenizer, MAX_LEN)

testing_set = EmailData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (441, 2)
TRAIN Dataset: (309, 2)
TEST Dataset: (132, 2)


In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,

                'shuffle': True,

                'num_workers': 0

                }



test_params = {'batch_size': VALID_BATCH_SIZE,

                'shuffle': True,

                'num_workers': 0

                }



training_loader = DataLoader(training_set, **train_params)

testing_loader = DataLoader(testing_set, **test_params)

In [15]:
class RobertaClass(torch.nn.Module):

    def __init__(self):

        super(RobertaClass, self).__init__()

        self.l1 = RobertaModel.from_pretrained("roberta-base")

        self.pre_classifier = torch.nn.Linear(768, 768)

        self.dropout = torch.nn.Dropout(0.3)

        self.classifier = torch.nn.Linear(768, 3)



    def forward(self, input_ids, attention_mask, token_type_ids):

        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        hidden_state = output_1[0]

        pooler = hidden_state[:, 0]

        pooler = self.pre_classifier(pooler)

        pooler = torch.nn.ReLU()(pooler)

        pooler = self.dropout(pooler)

        output = self.classifier(pooler)

        return output

In [16]:
model = RobertaClass()

model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [17]:
loss_function = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [18]:
def calcuate_accuracy(preds, targets):

    n_correct = (preds==targets).sum().item()

    return n_correct

In [19]:


def train(epoch):

    tr_loss = 0

    n_correct = 0

    nb_tr_steps = 0

    nb_tr_examples = 0

    model.train()

    for _,data in tqdm(enumerate(training_loader, 0)):

        ids = data['ids'].to(device, dtype = torch.long)

        mask = data['mask'].to(device, dtype = torch.long)

        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

        targets = data['targets'].to(device, dtype = torch.long)



        outputs = model(ids, mask, token_type_ids)

        loss = loss_function(outputs, targets)

        tr_loss += loss.item()

        big_val, big_idx = torch.max(outputs.data, dim=1)

        n_correct += calcuate_accuracy(big_idx, targets)



        nb_tr_steps += 1

        nb_tr_examples+=targets.size(0)

        

        if _%5000==0:

            loss_step = tr_loss/nb_tr_steps

            accu_step = (n_correct*100)/nb_tr_examples 

            print(f"Training Loss per 5000 steps: {loss_step}")

            print(f"Training Accuracy per 5000 steps: {accu_step}")



        optimizer.zero_grad()

        loss.backward()

        # # When using GPU

        optimizer.step()



    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')

    epoch_loss = tr_loss/nb_tr_steps

    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Training Loss Epoch: {epoch_loss}")

    print(f"Training Accuracy Epoch: {epoch_accu}")



    return 

In [20]:
EPOCHS = 5

for epoch in range(EPOCHS):

    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 1.130231261253357
Training Accuracy per 5000 steps: 0.0


155it [00:18,  8.55it/s]


The Total Accuracy for Epoch 0: 70.87378640776699
Training Loss Epoch: 0.6521657913442581
Training Accuracy Epoch: 70.87378640776699


2it [00:00, 10.60it/s]

...

The Total Accuracy for Epoch 3: 100.0
Training Loss Epoch: 0.005937779787927866
Training Accuracy Epoch: 100.0


0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.004064043518155813
Training Accuracy per 5000 steps: 100.0


155it [00:19,  8.06it/s]

The Total Accuracy for Epoch 4: 100.0
Training Loss Epoch: 0.003714523272168252
Training Accuracy Epoch: 100.0


In [21]:
def valid(model, testing_loader):

    model.eval()

    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

    with torch.no_grad():

        for _, data in tqdm(enumerate(testing_loader, 0)):

            ids = data['ids'].to(device, dtype = torch.long)

            mask = data['mask'].to(device, dtype = torch.long)

            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()

            loss = loss_function(outputs, targets)

            tr_loss += loss.item()

            big_val, big_idx = torch.max(outputs.data, dim=1)

            n_correct += calcuate_accuracy(big_idx, targets)



            nb_tr_steps += 1

            nb_tr_examples+=targets.size(0)

            

            if _%5000==0:

                loss_step = tr_loss/nb_tr_steps

                accu_step = (n_correct*100)/nb_tr_examples

                print(f"Validation Loss per 100 steps: {loss_step}")

                print(f"Validation Accuracy per 100 steps: {accu_step}")

    epoch_loss = tr_loss/nb_tr_steps

    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")

    print(f"Validation Accuracy Epoch: {epoch_accu}")

    

    return epoch_accu


In [22]:
acc = valid(model, testing_loader)

print("Accuracy on test data = %0.2f%%" % acc)

3it [00:00, 21.99it/s]

Validation Loss per 100 steps: 0.0008099359693005681
Validation Accuracy per 100 steps: 100.0


66it [00:02, 32.94it/s]

Validation Loss Epoch: 0.004147799305483755
Validation Accuracy Epoch: 100.0
Accuracy on test data = 100.00%


In [24]:

output_model_file = 'pytorch_roberta_email_state_dict.pt'
output_vocab_file = './'

model_to_save = model.state_dict()
torch.save(model_to_save, output_model_file)

tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')


All files saved
